In [ ]:
# install syft if we are on google colab
import sys, os; os.system("pip install --pre syft") if "google.colab" in sys.modules else ""
import syft as sy
print(f"You're running syft version: {sy.__version__}")

In [ ]:
domains = {} # our logged in domain clients
domain_credentials = set() # our set of domain credentials

In [ ]:
# enter the dict given from the data owner
def add_credentials(credentials_dict):
    fs = frozenset(credentials_dict.items())
    domain_credentials.add(fs)

In [ ]:
add_credentials(
    {'url': '52.157.8.193', 'name': 'Samantha Carter', 'email': 'sam@sg1.net', 'password': 'stargate'}
)

In [ ]:
domain_credentials

In [ ]:
def login_to_domains(force: bool = False):
    for fd in domain_credentials:
        credentials = dict(fd)
        if credentials["url"] not in domains or force:
            try:
                details = credentials.copy()
                del details["name"]
                del details["dataset_name"]
                client = sy.login(**details)
                domains[credentials["url"]] = client
            except Exception as e:
                print(e)

In [ ]:
login_to_domains()

In [ ]:
domains

In [ ]:
domains["52.157.8.193"].store